In [1]:
# imports
import os

import pandas as pd
import tensorflow as tf
import numpy as np
import mlflow

from mlflow.tracking import MlflowClient
from tensorflow import keras
from keras import layers

2024-04-29 12:29:08.973054: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:29:08.976583: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-29 12:29:09.023356: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-29 12:29:10.164114: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Here we add the path to the local module so we can import our own funtion to the notebook.

In [2]:
# import local modules
import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/sideProjects/deepLearning')
from deeplearning.deep_utils import imdb_model_setup

Now we load the `.csv` file with the runs from the previous exploration.

In [3]:
PATH_TO_RUNS = "/mnt/0A2AAC152AABFBB7/sideProjects/deepLearning/mlflow_runs"
old_runs = pd.read_csv(os.path.join(PATH_TO_RUNS, 'imdb_runs_1.csv'))

In [4]:
# Loading the IMDB dataset

(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.imdb.load_data(num_words=10000)

In [5]:
# Encoding the integer sequences via multi-hot encoding

def vectorize_sequences(sequences, dimension=10000):

    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        for j in sequence:
            results[i, j] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

In [6]:
# Setting aside a validation set

x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [7]:
# Initialize client and experiment

EXPERIMENT_NAME = "deepLearning_IMDB_dataset"
client = MlflowClient()
mlflow.set_experiment(EXPERIMENT_NAME)
exp = client.get_experiment_by_name(EXPERIMENT_NAME)

In [8]:
units_2_hidden = [(16, 16), (32, 32), (64,64)]

In [9]:
count = 0
for units in units_2_hidden:

    model, history = imdb_model_setup(
        units_layer1=units[0],
        units_layer2=units[1],
        activ_func="tanh",
        X_train=partial_x_train,
        y_train=partial_y_train,
        X_val=x_val,
        y_val=y_val
    )
    hist_dict = history.history
    test_loss, test_accuracy = model.evaluate(x_test, y_test)


    with mlflow.start_run():
        mlflow.set_tag("model", "Base_tanh_{}".format(count))
        mlflow.log_param("units_1st_layer", units[0])
        mlflow.log_param("units_2nd_layer", units[1])
        mlflow.log_param("hidden_activation", 'tanh')
        mlflow.log_param("epochs", 4)
        mlflow.log_param("batch_size", 512)
        mlflow.log_metric("accuracy", test_accuracy)
        mlflow.log_metric("loss", test_loss)
    count+=1

    del hist_dict
    del history
    del model

Epoch 1/4


2024-04-29 12:29:23.057364: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


26/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6942 - loss: 0.5950 

2024-04-29 12:29:26.310573: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 68ms/step - accuracy: 0.7094 - loss: 0.5816 - val_accuracy: 0.8433 - val_loss: 0.4096
Epoch 2/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8937 - loss: 0.3331 - val_accuracy: 0.8798 - val_loss: 0.3148
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9191 - loss: 0.2409 - val_accuracy: 0.8892 - val_loss: 0.2765
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.9401 - loss: 0.1817 - val_accuracy: 0.8891 - val_loss: 0.2718
 92/782 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8859 - loss: 0.2738

...

782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8793 - loss: 0.2899
Epoch 1/4


2024-04-29 12:29:34.579988: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 600000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6882 - loss: 0.5834

2024-04-29 12:29:36.190044: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 400000000 exceeds 10% of free system memory.


30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.6913 - loss: 0.5803 - val_accuracy: 0.8110 - val_loss: 0.4165
Epoch 2/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.8929 - loss: 0.2916 - val_accuracy: 0.8850 - val_loss: 0.2820
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9198 - loss: 0.2082 - val_accuracy: 0.8880 - val_loss: 0.2729
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.9484 - loss: 0.1469 - val_accuracy: 0.8859 - val_loss: 0.2887
782/782 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8741 - loss: 0.3132
Epoch 1/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.6929 - loss: 0.5700 - val_accuracy: 0.8393 - val_loss: 0.3702
Epoch 2/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - accuracy: 0.8935 - loss: 0.2725 - val_accuracy: 0.8555 - val_loss: 0.3530
Epoch 3/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9005 - loss: 0.2393 - val_accuracy: 0.8750 - val_loss: 0.3127
Epoch 4/4
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - a

In [14]:
new_runs =mlflow.search_runs()
new_runs[['experiment_id', 'status', 'metrics.loss', 'metrics.accuracy', 'params.units_1st_layer', 'params.units_2nd_layer', "params.hidden_activation"]]

,experiment_id,status,metrics.loss,metrics.accuracy,params.units_1st_layer,params.units_2nd_layer,params.hidden_activation
0,926124755463801135,FINISHED,0.338135,0.87084,64,64,tanh
1,926124755463801135,FINISHED,0.309813,0.87840,32,32,tanh
2,926124755463801135,FINISHED,0.287834,0.88152,16,16,tanh
3,926124755463801135,FINISHED,0.385586,0.84884,64,64,relu
4,926124755463801135,FINISHED,0.313741,0.87392,32,32,relu
5,926124755463801135,FINISHED,0.294502,0.88060,16,16,relu


In [15]:
runs = pd.concat([old_runs, new_runs], axis=0)
runs[['experiment_id', 'status', 'metrics.loss', 'metrics.accuracy', 'params.units_1st_layer', 'params.units_2nd_layer', "params.hidden_activation"]]

,experiment_id,status,metrics.loss,metrics.accuracy,params.units_1st_layer,params.units_2nd_layer,params.hidden_activation
0,926124755463801135,FINISHED,0.385586,0.84884,64,64,relu
1,926124755463801135,FINISHED,0.313741,0.87392,32,32,relu
2,926124755463801135,FINISHED,0.294502,0.88060,16,16,relu
0,926124755463801135,FINISHED,0.338135,0.87084,64,64,tanh
1,926124755463801135,FINISHED,0.309813,0.87840,32,32,tanh
2,926124755463801135,FINISHED,0.287834,0.88152,16,16,tanh
3,926124755463801135,FINISHED,0.385586,0.84884,64,64,relu
4,926124755463801135,FINISHED,0.313741,0.87392,32,32,relu
5,926124755463801135,FINISHED,0.294502,0.88060,16,16,relu


In [12]:
runs.to_csv(
    os.path.join(PATH_TO_RUNS, 'imdb_runs_1.csv')
)